In [ ]:
import requests

# Replace with your Google API key
GOOGLE_API_KEY = '<enter your google api key here>'

# Define the URL for the Google Cloud Natural Language API
url = "https://language.googleapis.com/v2/documents:moderateText"

# Set the headers with the Google API key and content type
headers = {
    "X-Goog-Api-Key": GOOGLE_API_KEY,
    "Content-Type": "application/json",
}

In [ ]:
from multiprocessing import Manager
from threading import Thread
from time import sleep

manager = Manager()

from tqdm import tqdm
import pickle

with open('./results/comments.pkl', 'rb') as handle:
    comments = pickle.load(handle)

comments = list(set(comments.values()))

moderation_map = manager.dict()

def moderation(local_dictionary, texts):
    for comment in tqdm(texts):
        try:
            # Define the JSON data to send in the request
            data = {
                "document": {
                    "content": comment,
                    "type": "PLAIN_TEXT",
                }
            }

            # Make the POST request to the API
            response = requests.post(url, headers=headers, json=data)

            local_dictionary[comment] = response.json()

        except Exception as e:
            local_dictionary[comment] = None

In [ ]:
num_threads = 5
threads = []
for i in range(0, num_threads):
    start = i * (len(comments) // num_threads)
    end = (i + 1) * (len(comments) // num_threads) if i != num_threads - 1 else len(comments)
    thread = Thread(target=moderation,args=(moderation_map, comments[start:end], ))
    threads += [thread]
    thread.start()

for thread in threads:
    thread.join()

In [ ]:
with open('./results/comments_moderation_google_palm.pkl', 'wb') as handle:
    pickle.dump(dict(moderation_map), handle, protocol=pickle.HIGHEST_PROTOCOL)